In [ ]:
# Put images into 3 class inside TestData folder

import os
test_data_dir = 'TestData'
# print all class in the test 
test = os.listdir(test_data_dir)
print(test) 

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

# converting images to tensors and normalizing them without applying data augmentation 
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# apply transform on dataset
test_set = ImageFolder(root=test_data_dir, transform=test_transforms)

# load data to loader
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

# check the size after transfer to loader for Torch
print(f"Number of training samples: {len(test_set)}")
# Check the shape of the first training image [channel, width, height]
test_x, tset_y = test_set[0]
print(f"Shape of first training images in first batch : {test_x.shape}")
for batch in test_loader:
    images, labels = batch
    print(f"Shape of first training images in first batch: {images.shape}")  # [batch_size, channels, height, width]
    print(f"Check type of input and label", type(images), type(labels))
    break  

### CNN model

In [ ]:
# mport pytorch library and modules
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.optim as optim
from torch.optim import Adam
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, BatchNorm2d, BatchNorm1d, Dropout

class convnet(nn.Module):
    # constructor initialize instances of class
    def __init__(self, num_classes):
        super(convnet, self).__init__()        

        self.cnn_layers = Sequential(
            
            Conv2d(3, 64, kernel_size = 3, padding = 1),  
            BatchNorm2d(64),                                       
            ReLU(),                                                 
            Dropout(p=0.25),                                        
            MaxPool2d(kernel_size=2),                               

            # second block
            Conv2d(64, 128, kernel_size = 5, padding = 1),            
            BatchNorm2d(128), 
            ReLU(),
            Dropout(p=0.25),
            MaxPool2d(kernel_size=2),
            
            # third block
            Conv2d(128, 512, kernel_size = 3, padding = 1),           
            BatchNorm2d(512), 
            ReLU(),                                                
            Dropout(p=0.25),
            MaxPool2d(kernel_size=2),

            # fourth block
            Conv2d(512, 512, kernel_size = 3, padding = 1),          
            BatchNorm2d(512), 
            ReLU(),
            Dropout(p=0.25),
            MaxPool2d(kernel_size=2),
        )


        # definind 2 fully connected layer (multi layers nn)
        self.linear_layers = Sequential(
          
            # first fully connected layer  (input to hidden1)
            Linear(512 * 13 * 13, 256),      
            BatchNorm1d(256),
            ReLU(),
            Dropout(p=0.5),

            # second fully connected layer  (hidden to hidden 2)
            Linear(256, 512),                 
            BatchNorm1d(512),
            ReLU(),
            Dropout(p=0.5),

            # hidden to output
            Linear(512, num_classes)
        )
    
    def forward(self, x):
        # forward pass through cnn and linear layers
        z1 = self.cnn_layers(x)  
        z1 = z1.view(z1.size(0), -1)   
        z2 = self.linear_layers(z1) 
        
        # return F.softmax(z2, dim = 1) 
        return z2

In [ ]:
# Instantiate the model
num_classes = 3
model = convnet(num_classes)

# defining the optimizer: Adam with regulation to prevent overfitting
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay=1e-7)      
# defining the loss function
criterion = CrossEntropyLoss()
print(model)


In [ ]:
# For MAC user
# device = "mps" if torch.backends.mps.is_available() else "cpu"
# print(device)
# model = model.to(device)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = model.to(device)

In [ ]:

# use for function call during the training process to cal + print the accuracy in each epoch
def calculate_accuracy(model, data_loader):
    model.eval()  
    predictions = 0
    total_samples = 0
    true_labels = []
    pred_labels = []

    with torch.no_grad():  # No track gradients
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            y_pred = torch.argmax(outputs, dim=1)                  

            # accuracy metric as the number of correctly predicted results / total numbe of samples
            predictions += torch.sum(y_pred == labels).item()
            total_samples += labels.size(0)                        

            pred_labels.extend(y_pred.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    total_acc = predictions / total_samples
    
    return total_acc, true_labels, pred_labels

In [ ]:
test_accuracy, test_true_labels, test_pred_labels = calculate_accuracy(model, test_loader)
print(f'Test final perfomance Accuracy: {test_accuracy:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

class_names = ['Class 3 No Touch w Hands', 'Class 1 Touch', 'Class 2 No hands']

# Generate confusion matrix
conf_matrix = confusion_matrix(test_true_labels, test_pred_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix with class names
plt.figure(figsize=(12, 6))
sns.heatmap(conf_matrix_normalized, annot=True, cmap="Blues", fmt=".2f", cbar=True, 
            xticklabels=class_names, yticklabels=class_names)

plt.title("Normalized confusion matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()